# ResNET CNN
In papers describes as many layer stack in network Accuracy will increase which is not true. <br>
As we increase layers in vanish gradient problem occure.<br>
Gradient decent decrease exponentially as we backpropogate. <br>
As we multiply many weight gradient reach toward Zero which cause early layers weight will change very small amount. <br>
Here is example: gradient = w[l-1] * w[l-2] * w[l] ......... w[l] <br>
if we multiply small numbers < 1 it create another small number.
## ResNET
ResNET solve this issue by adding "skip connection".
Skip Connection create a small path.


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor

2024-10-20 22:41:18.867303: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 22:41:19.978488: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 22:41:20.170059: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 22:41:21.738528: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-20 22:41:34.686044: W tensorflow/compiler/tf2

# Identity Block
implementation of Identity Block

In [2]:
def identity_block(X, f, filters, training = True, initializer = random_uniform):
    """
    X --> is input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f --> integer specifying middle conv layer window of main path.
    filters --> python list of integers, defining the number of filters in Conv layer in main path.
    training --> true: behaveing in training mode.
                 False : behave not in training mode.
    initializer--> set up initial weight of layer using random uniform.
    """
    
    F1, F2, F3 = filters 
    X_shortcut = X
    cache = []
    X = Conv2D(F1, kernel_size = 1, strides = (1, 1), padding = "valid", kernel_initializer = initializer(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation("relu")(X)
    
    X =  Conv2D(F2, kernel_size = f, strides = (1, 1), padding = "same", kernel_initializer = initializer(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation("relu")(X)
    
    X = Conv2D(F3, kernel_size = 1, strides = (1, 1), padding = "valid", kernel_initializer = initializer(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    
    X = Add()([X_shortcut, X])
    
    X = Activation("relu")(X)
    
    return X
    

In [3]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)
A3 = identity_block(X, f=2, filters=[4, 4, 3], training = False, initializer=lambda seed=0:constant(value=1))
print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()
print(np.around(A3.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))
resume = A3np[:,(0,-1),:,:].mean(axis = 3)
print(resume[1, 1, 0])

print('\n\033[1mWith training=True\033[0m\n')
np.random.seed(1)
A4 = identity_block(X, f=2, filters=[3, 3, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=True)
print(np.around(A4.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))

With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.71236 192.71236 192.71236  96.85619]
  [ 96.85619  96.85619  96.85619  48.9281 ]]

 [[578.1371  578.1371  578.1371  290.56854]
  [290.56854 290.56854 290.56854 146.78427]]]
96.85619

With training=True

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40739 0.40739 0.40739 0.40739]
  [0.40739 0.40739 0.40739 0.40739]]

 [[4.99991 4.99991 4.99991 3.25948]
  [3.25948 3.25948 3.25948 2.40739]]]


# Convolve Layer
In this layer we are going to implement convlove layer between shortcut path that helps to change input dimensions that are equal to main path dimension we called this block to Convolve block


In [24]:
def convolve_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    """
    X --> Input tensor dimensions (m, n_h_prev, n_w_prev, n_c_prev)
    f --> size of kernal of used in layer
    filters --> numbers of filter and kernal used in convlove layer
    training --> if training is model going to train else for prediction
    """
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(F1, kernel_size = 1, strides = (s, s), padding = "valid", kernel_initializer = initializer(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation("relu")(X)
    
    X = Conv2D(F2, kernel_size = f, strides = (1, 1), padding = "same", kernel_initializer = initializer(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation("relu")(X)
    
    X = Conv2D(F3, kernel_size = 1, strides = (1, 1), padding = "valid", kernel_initializer = initializer(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    
    X_shortcut = Conv2D(F3, kernel_size = 1, strides = (s, s), padding = "valid", kernel_initializer = initializer(seed = 0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training = training)
    
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [26]:
np.random.seed(1)
#X = np.random.randn(3, 4, 4, 6).astype(np.float32)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)
A = convolve_block(X, f = 2, filters = [2, 4, 6], training=False)

assert type(A) == EagerTensor, "Use only tensorflow and keras functions"
assert tuple(tf.shape(A).numpy()) == (3, 2, 2, 6), "Wrong shape."
print(A[0])

tf.Tensor(
[[[0.         0.66683817 0.         0.         0.888539   0.5274254 ]
  [0.         0.65053666 0.         0.         0.8959285  0.49965227]]

 [[0.         0.6312079  0.         0.         0.86362475 0.47643146]
  [0.         0.56883204 0.         0.         0.8553412  0.417093  ]]], shape=(2, 2, 6), dtype=float32)


In [ ]:
def ResNet_50(input_shape = (64, 64, 3), classes = 6):
    x_input = Input(input_shape)
    
    X = ZeroPadding3D((3, 3))(x_input)
    X = Conv2D(64, kernel_size = 7, strides = (2, 2), name = "conv1", kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = "bnconv1")(X)
    X MaxPooling2D((3, 3), strides = (2, 2))(X)
    
    X = convolve_block(X, f = 3, [64, 64, 256], s = 1)
    X = identity_block(X, )
    